In [1]:
import os
os.chdir("..")

In [2]:
from torch.nn import Module
from src.experiment import (
    CLIPGraphRun,
    CLIPRun,
)
from src.utils import load_ruamel
import torch

/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def count_parameters(model: Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# GraphCLIP

In [4]:
root = "./configs_cineca/proposed_model"

In [5]:
params_data = dict()
for task in ("style", "genre"):
    for conf in filter(lambda x: x.startswith("normal"), os.listdir(f"{root}/{task}/")):
        run_cls = CLIPGraphRun
        params = load_ruamel(f"{root}/{task}/{conf}")
        params["clean_out_dir"] = False
        run = run_cls(params)
        run.set_model_parameters_for_warmup(current_epoch=0)
        params_data[f"{task}-{conf}"] = count_parameters(run.model)

Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!
Warm-up phase. ViT completely frozen
Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!
Warm-up phase. ViT completely frozen
Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!
Warm-up phase. ViT completely frozen
Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
Done!
Loading metrics...
Done!
Lo

In [6]:
{k: f"{v:.3e}" for k, v in params_data.items()}

{'style-normal_clip_graph_sage3.yaml': '6.140e+07',
 'style-normal_clip_graph_gat2.yaml': '1.641e+08',
 'style-normal_clip_graph_sage2.yaml': '6.142e+07',
 'style-normal_clip_graph_gat3.yaml': '2.461e+08',
 'genre-normal_clip_graph_sage3.yaml': '6.140e+07',
 'genre-normal_clip_graph_gat2.yaml': '1.641e+08',
 'genre-normal_clip_graph_sage2.yaml': '6.142e+07',
 'genre-normal_clip_graph_gat3.yaml': '2.461e+08'}

# AIxIA

In [7]:
from src.experiment import AIxIAMultiTaskRun, AIxIARun

root = "./configs_cineca/baselines"
params_data = dict()
for task in ("style", "genre", "multitask"):
    for conf in filter(
        lambda x: x.startswith("normal_aixia"), os.listdir(f"{root}/{task}/")
    ):
        print(f"{task}/{conf}")
        run_cls = AIxIAMultiTaskRun if task == "multitask" else AIxIARun
        params = load_ruamel(f"{root}/{task}/{conf}")
        params["clean_out_dir"] = False
        run = run_cls(params)
        # lazy initialization
        run.model(
            torch.randn(1, 3, 224, 224).to("cuda"),
            run.graph.x_dict,
            run.graph.edge_index_dict,
        )
        for p in run.model.vit.parameters():
            p.requires_grad = False
        for p in run.model.vit.head.parameters():
            p.requires_grad = True
        params_data[f"{task}-{conf}"] = count_parameters(run.model)

style/normal_aixia.yaml
Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...


loading state dict for vit


/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!
genre/normal_aixia.yaml
Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
loading state dict for vit
Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!
multitask/normal_aixia.yaml
Loading general parameters...


/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Done!
Loading dataloaders...
Done!
Loading model...
loading state dict for vit


/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!


In [8]:
{k: f"{v:.3e}" for k, v in params_data.items()}

{'style-normal_aixia.yaml': '1.227e+07',
 'genre-normal_aixia.yaml': '9.070e+06',
 'multitask-normal_aixia.yaml': '2.878e+07'}

# PREVIOUS KBS

In [9]:
from src.models.models_kg import (
    ContextNetSingleTask,
    ContextNetlMultiTask,
    NewMultiModalMultiTaskViT,
    NewMultiModalSingleTaskVit,
    LabelProjectorVit,
)

In [10]:
emb_size = 128

In [11]:
def count_previous_kbs(emb_size, num_class):
    model_cls = (
        NewMultiModalMultiTaskViT
        if isinstance(num_class, dict)
        else NewMultiModalSingleTaskVit
    )
    model = model_cls(emb_size=emb_size, num_class=num_class, dropout=0.0)
    aixia_run = AIxIARun(load_ruamel('./configs_cineca/baselines/genre/normal_aixia.yaml'))
    gnn = aixia_run.model.gnn
    with torch.no_grad():
        gnn(aixia_run.graph.x_dict, aixia_run.graph.edge_index_dict)
    projector = LabelProjectorVit(emb_size=emb_size)
    return count_parameters(model) + count_parameters(gnn) + count_parameters(projector)


In [12]:
{
    "kbs-style": f"{count_previous_kbs(emb_size=128, num_class=32):.3e}",
    "kbs-genre": f"{count_previous_kbs(emb_size=128, num_class=18):.3e}",
    "kbs-multitask": f"{count_previous_kbs(emb_size=128, num_class=dict(style=32, genre=18)):.3e}"
}

Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
loading state dict for vit
Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!


/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
loading state dict for vit
Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!


/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Loading general parameters...
Done!
Loading dataloaders...
Done!
Loading model...
loading state dict for vit
Done!
Loading metrics...
Done!
Loading tokenizer...
Done!
Loading criterion...
Done!
Loading optimizer...
Done!
Loading scheduler...
Done!
Loading early stop callback...
Done!


/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'kbs-style': '1.234e+07',
 'kbs-genre': '1.233e+07',
 'kbs-multitask': '1.236e+07'}

In [13]:
def count_contextnet(emb_size, num_class) -> ContextNetSingleTask | ContextNetlMultiTask:
    model_cls = (
        ContextNetlMultiTask if isinstance(num_class, dict) else ContextNetSingleTask
    )
    model = model_cls(emb_size=emb_size, num_class=num_class)
    for p in model.resnet.parameters():
        p.requires_grad = False
    for p in model._modules["resnet"]._modules["7"].parameters():
        p.requires_grad = True
    return count_parameters(model) + emb_size*59339

In [14]:
{
    "contextnet-style": f"{count_contextnet(emb_size=128, num_class=32):.3e}",
    "contextnet-genre": f"{count_contextnet(emb_size=128, num_class=18):.3e}",
    "contextnet-multitask": f"{count_contextnet(emb_size=128, num_class=dict(style=32, genre=18)):.3e}"
}

/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/raffaele/art/contrastive-artwork-classification/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


{'contextnet-style': '2.289e+07',
 'contextnet-genre': '2.286e+07',
 'contextnet-multitask': '2.292e+07'}